# Visualisation & Evaluation

In [ ]:
import os
from typing import List
import torch 
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
from labml import lab, tracker, experiment, monit
from labml.configs import BaseConfigs, option
from labml_helpers.device import DeviceConfigs
import numpy as np
import matplotlib.pyplot as plt
import time
from torch.nn.parallel.distributed import DistributedDataParallel as DDP
import torch.distributed as dist

In [ ]:
# # # Below are not relevant for training purpose.   

# def normalize_tensor(tensor, value_range=None):
#     tensor = tensor.clone()  # avoid modifying tensor in-place
#     if value_range is not None and not isinstance(value_range, tuple):
#         raise TypeError("value_range has to be a tuple (min, max) if specified. min and max are numbers")

#     def norm_ip(img, low, high):
#         img.clamp_(min=low, max=high)
#         img.sub_(low).div_(max(high - low, 1e-5))

#     def norm_range(t, value_range):
#         if value_range is not None:
#             norm_ip(t, value_range[0], value_range[1])
#         else:
#             norm_ip(t, float(t.min()), float(t.max()))

#     for t in tensor:
#         norm_range(t, value_range)
#     return tensor


# tensor = torch.load('samples.pt')
# norm_tensor = normalize_tensor(tensor)
# torch.save(norm_tensor, 'samples.pt')

In [ ]:
tensor = torch.load('samples_bbb.pt')
# s_tensor = torch.cat([tensor] * 30, dim=0)
# print(s_tensor.shape)
# torch.save(s_tensor, 'l_samples.pt')
print(tensor.shape)
print(torch.max(tensor))

tensor = torch.load('dataset.pt')
print(tensor.shape)
print(torch.max(tensor))

# # Below are not relevant for training purpose.   

# def normalize_tensor(tensor, value_range=None):
#     tensor = tensor.clone()  # avoid modifying tensor in-place
#     if value_range is not None and not isinstance(value_range, tuple):
#         raise TypeError("value_range has to be a tuple (min, max) if specified. min and max are numbers")

#     def norm_ip(img, low, high):
#         img.clamp_(min=low, max=high)
#         img.sub_(low).div_(max(high - low, 1e-5))

#     def norm_range(t, value_range):
#         if value_range is not None:
#             norm_ip(t, value_range[0], value_range[1])
#         else:
#             norm_ip(t, float(t.min()), float(t.max()))

#     for t in tensor:
#         norm_range(t, value_range)
#     return tensor


# tensor = torch.load('samples_bbb.pt')
# norm_tensor = normalize_tensor(tensor)
# torch.save(norm_tensor, 'samples_bbb.pt')


# tensor_n = tensor[0:16, :, :, :]
# print(tensor_n.shape)
# torch.save(tensor_n, 'tensor_n.pt')


# Compare gen samples with true samples

In [ ]:
# Below are not relevant for training purpose.   
# Display the sampled images
import matplotlib.pyplot as plt
import numpy as np
import torch

tensor = torch.load('samples.pt')
print('gen_shape:', tensor.shape)
N = tensor.size(0)

# Create a grid of images from the tensor
grid = torchvision.utils.make_grid(tensor, nrow=int(N**(0.5)), padding=1, normalize=True, pad_value=1.0, scale_each=True)

# Convert the grid to a numpy array and transpose the dimensions to match the expected format by matplotlib
grid = grid.cpu().numpy().transpose((1, 2, 0))

# Show the grid of images using matplotlib
plt.imshow(grid)
plt.axis('off')
plt.savefig('gen_samples.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()



# # display the true images
tensor = torch.load('true_samples.pt')
print('true_shape:', tensor.shape)
N = tensor.size(0)

# Create a grid of images from the tensor
grid = torchvision.utils.make_grid(tensor, nrow=int(N**(0.5)), padding=1, normalize=True, pad_value=1.0)

# Convert the grid to a numpy array and transpose the dimensions to match the expected format by matplotlib
grid = grid.cpu().numpy().transpose((1, 2, 0))

# Show the grid of images using matplotlib
plt.imshow(grid)
plt.axis('off')
plt.savefig('true_samples.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.show()



# Compare two plots
# Load the saved images
gen_samples = plt.imread('gen_samples.png')
true_samples = plt.imread('true_samples.png')

# Create a new figure with two subplots side by side
fig, axs = plt.subplots(1, 2, figsize=(18, 9), facecolor='white')

# Plot the generated samples on the left subplot
axs[0].imshow(gen_samples)
axs[0].set_title('Generated Samples', fontsize=20)
axs[0].axis('off')


# Plot the true samples on the right subplot
axs[1].imshow(true_samples)
axs[1].set_title('True Samples', fontsize=20)
axs[1].axis('off')


# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.1)

# Display the figure
plt.savefig('Gen_vs_True_CIFAR10.png', dpi=300)
plt.show()

# Compute Inception Score & FID score

In [ ]:
# import torch

# import sys
# original_path = sys.path.copy()  # make a copy of the original sys.path

# # remove the system-wide dist-packages directory from sys.path
# sys.path = ['/home/research/yuqijing/sem2project/Diffusion',
#     '/home/research/yuqijing/.local/lib/python3.9/site-packages/git/ext/gitdb',
#     '/usr/lib/python39.zip',
#     '/usr/lib/python3.9',
#     '/usr/lib/python3.9/lib-dynload',
#     '',
#     '/home/research/yuqijing/.local/lib/python3.9/site-packages',
#     '/usr/local/lib/python3.9/dist-packages',
#     '/usr/lib/python3/dist-packages',
#     '/usr/lib/python3/dist-packages/IPython/extensions',
#     '/home/research/yuqijing/.ipython']

# from torchmetrics.image.inception import InceptionScore
# # from /home/research/yuqijing/.local/lib/python3.9/site-packages import torchmetrics


# print(sys.path)



# inception = InceptionScore(normalize=True)
# # load images
# imgs = torch.load('samples.pt')
# imgs = imgs.cpu()
# inception.update(imgs)
# inception.compute()
# print(inception.compute())

# # add the system-wide dist-packages directory back to sys.path
# sys.path = original_path  # restore sys.path to its original state

In [ ]:
import torch

import sys
original_path = sys.path.copy()  # make a copy of the original sys.path

# remove the system-wide dist-packages directory from sys.path
sys.path = ['/home/research/yuqijing/sem2project/Diffusion',
    '/home/research/yuqijing/.local/lib/python3.9/site-packages/git/ext/gitdb',
    '/usr/lib/python39.zip',
    '/usr/lib/python3.9',
    '/usr/lib/python3.9/lib-dynload',
    '',
    '/home/research/yuqijing/.local/lib/python3.9/site-packages',
    '/usr/local/lib/python3.9/dist-packages',
    '/usr/lib/python3/dist-packages',
    '/usr/lib/python3/dist-packages/IPython/extensions',
    '/home/research/yuqijing/.ipython']

### Inception Score
from torchmetrics.image.inception import InceptionScore
# from /home/research/yuqijing/.local/lib/python3.9/site-packages import torchmetrics
start_time = time.time()
inception = InceptionScore(feature=64, normalize=True)
# load images
imgs = torch.load('data_small.pt')
imgs = imgs.cpu()
inception.update(imgs)
inception.compute()
end_time = time.time()
elp_time = end_time - start_time
print(inception.compute())
print('compute time:', elp_time, 'seconds')

# ### FID
# from torchmetrics.image.fid import FrechetInceptionDistance
# start_time = time.time()
# fid = FrechetInceptionDistance(feature=2048, normalize=True)
# # load images
# imgs_1 = torch.load('dataset.pt')
# imgs_1 = imgs_1.cpu()
# imgs_2 = torch.load('samples_bbb.pt')
# print(torch.max(imgs_1))
# imgs_2 = imgs_2.cpu()
# imgs_dist1 = imgs_1
# imgs_dist2 = imgs_2
# fid.update(imgs_dist1, real=True)
# fid.update(imgs_dist2, real=False)
# fid.compute()
# end_time = time.time()
# elp_time = end_time - start_time
# print(fid.compute())
# print('compute time:', elp_time, 'seconds')

# for reference, the true cifar10 achieves IS of 11.24 +/- 0.12
# for reference, the best generative model (score-based + diffusion model) achieves FID of 1.64 (lowest).





# add the system-wide dist-packages directory back to sys.path
sys.path = original_path  # restore sys.path to its original state

In [ ]:
# FID & IS for CIFAR10 dataset

import torch
from pytorch_gan_metrics import get_inception_score, get_fid

images = torch.load('/home/research/yuqijing/sem2project/Diffusion/FID_statistics/data_small.pt')
print(images.shape)
print(torch.max(images))
# IS calculation
IS, IS_std = get_inception_score(images)
print(IS, IS_std)
# FID calculation
FID = get_fid(images, './FID_statistics/statistics_c.npz') # Frechet Inception Distance
print(FID)



In [ ]:
# Convert 1 channel images to 3 channel images for MNIST images.
images = torch.load('samples_bbb.pt')
print(images.shape)
print(images[3, :, 10, 10])
images_2 = torch.load('true_samples_bbb.pt')
print(images_2.shape)

images_3 = torch.cat([images, images, images], dim=1)
print(images_3.shape)
print(images_3[3, :, 10, 10])
images_4 = torch.cat([images_2, images_2, images_2], dim=1)
print(images_4.shape)
torch.save(images_4, 'true_samples_m.pt')
torch.save(images_3, 'samples_m.pt')

# IS for MNIST

In [ ]:
# Calculate IS for the MNIST images. 
import torch
from pytorch_gan_metrics import get_inception_score, get_fid

images = torch.load('true_samples_m.pt')
print(images.shape)
print(torch.max(images))
# IS calculation
IS, IS_std = get_inception_score(images)
print(IS, IS_std)

# IS for the 10_000 true MNIST images are 2.004
# IS for the 10_000 generated MNIST images are 1.8745



# FID for MNIST please refer to stf in sem2project

# For creating reference dataset dataset.pt, 10,000, 3, 32, 32

In [ ]:
# # tensor = np.load('cifar10-32x32.npz')

# # print(tensor.files)
# tensor = torch.load('samples.pt')
# tensor.shape

# Sample Images from the true density
# Sample Images from the true density
import random
N = 10_000
    
# Define the transform to apply to each image
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((32, 32))
])

# Load the CIFAR10 dataset with the defined transform
dataset_f = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Get the indices of 16 randomly sampled images from the dataset
indices = random.sample(range(len(dataset_f)), N)

# Create a tensor to hold the 16 images
tensor = torch.zeros((N, 3, 32, 32))

# Load each sampled image into the tensor
for i, idx in enumerate(indices):
    image, _ = dataset_f[idx]
    tensor[i] = image

torch.save(tensor, 'dataset.pt')

# tensor = torch.load('dataset.pt')
# print(tensor.shape)


# Drawing samples from dataset.pt which is 10,000 true images from CIFAR10. 

In [ ]:
# # tensor = np.load('cifar10-32x32.npz')

# # print(tensor.files)
# tensor = torch.load('samples.pt')
# tensor.shape

# Sample Images from the true density
# Sample Images from the true density
import random
N = 1000
    
# Define the transform to apply to each image
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((32, 32))
])

# Load the CIFAR10 dataset with the defined transform
dataset_f = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Get the indices of 16 randomly sampled images from the dataset
indices = random.sample(range(len(dataset_f)), N)

# Create a tensor to hold the 16 images
tensor = torch.zeros((N, 3, 32, 32))

# Load each sampled image into the tensor
for i, idx in enumerate(indices):
    image, _ = dataset_f[idx]
    tensor[i] = image

torch.save(tensor, 'data_small.pt')   # 'data_small.pt' is a set of images of the form of tensor [N, 3, 32, 32]

# tensor = torch.load('dataset.pt')
# print(tensor.shape)


# Convert .pt to 10,000 number of images in one file

In [ ]:
from torchvision.utils import save_image
import os

# Set the name of the new directory
directory_name = "images_true"

# Create the new directory if it doesn't already exist
if not os.path.exists(directory_name):
    os.mkdir(directory_name)
    

tensor = torch.load('true_samples_m.pt')
print('samples:', tensor.shape)
N = tensor.size(0)
print(N)




for i in range(10_000):
    
    image = tensor[i]
    
    # Set the path where the new image file will be saved
    new_image_path = os.path.join(directory_name, f"image {i}.jpg")

    # Save the image to the new directory
    save_image(image, new_image_path)

# for i in range()



5.719s == 2048, 2.7s == 192, 5.5s == no, 4.8s == 768

# Install packages in specific path

In [ ]:
# #!pip install --target=/home/local/lib/python3.9/dist-packages/ torch-fidelity
# # !pip install --no-deps --target=/home/research/yuqijing/.local/lib/python3.9/site-packages torchmetrics
# !pip install --no-deps --target=/home/research/yuqijing/sem2project/Diffusion torch-fidelity

In [ ]:
# !rm -rf logs/